This notebook showcases the pipeline for linking entities in a table to wikidata. For the purpose of this tutorial, I use a small table.<br> 
The steps involved in this notebook are as follows:<br>
    a) Generate candidates using fuzzy augmented method and exact matches method present in table linker<br>
    b) Rank the candidates using centroid-singleton strategy using table linker commands. 

In [1]:
import pandas as pd
import os
import glob

In [5]:
table_path = '/data/candidate_generation/wikidata_es/t2dv2-canonical/table_linker_pipeline_data/'
table_name = 'players.csv'

In [6]:
p = os.path.join(table_path,table_name)
pd.read_csv(p,nrows = 10)

,Rank,Player (2011 TBs),TBs
0,1,Hank Aaron,"6,856"
1,2,Stan Musial,"6,134"
2,3,Willie Mays,"6,066"
3,4,Barry Bonds,"5,976"
4,5,Ty Cobb,"5,854"
5,6,Babe Ruth,"5,793"
6,7,Pete Rose,"5,752"
7,8,Carl Yastrzemski,"5,539"
8,9,Eddie Murray,"5,397"
9,10,Rafael Palmeiro,"5,388"


We will use this table and link the column "Player (2011 TBs)" to wikidata. <br>

Before start the process of candidate generation, we need to convert the table into a canonical format

### Canonicalize the input table

In [8]:
canonical_file = os.path.join(table_path,'canonical-players.csv')
!tl canonicalize -c "Player (2011 TBs)" $p > $canonical_file

In [9]:
pd.read_csv(canonical_file,nrows = 10)

,column,row,label
0,1,0,Hank Aaron
1,1,1,Stan Musial
2,1,2,Willie Mays
3,1,3,Barry Bonds
4,1,4,Ty Cobb
5,1,5,Babe Ruth
6,1,6,Pete Rose
7,1,7,Carl Yastrzemski
8,1,8,Eddie Murray
9,1,9,Rafael Palmeiro


### Candidate Generation

In [10]:
cand_file = os.path.join(table_path,'cand-players.csv')

In [11]:
%%time
!tl clean -c label -o label_clean $canonical_file / get-fuzzy-augmented-matches -c label_clean > $cand_file

CPU times: user 169 ms, sys: 67.2 ms, total: 236 ms
Wall time: 11.5 s
